# Rust Without the Standard Library

- Rust is intended to be a language for systems programming
- This chapter shows how to use Rust without the operating system

## Opting Out of the Standard Library

- The `core` library doesn’t depend on the operating system
- `alloc` depends on dynamic memory allocation
- The `std` library depends on `core`, `alloc` and the operating system
- The `#![no_std]` attribute allows to opt-out of everything but core language features
  - single `std` function can be still used via `extern std`

## Dynamic Memory Allocation

- dynamic memory is very flexible as it supports allocating variably sized regions
- `Vec`, `String`, `Arc` and `Rc` rely on dynamic memory
- By default Rust uses the system allocator
- `#![no_std]` excludes types that rely on dynamic memory
- Provided a custom allocator implementation, some collections are available from the `alloc` crate
  - use attribute `global_allocator` to provide an allocator
- Alternatively, heapless collections are just large enough preallocated vectors on the stack

## The Rust Runtime

- Rust lacks typical runtime features like the garbage collector
- But it brings the functionalities below

### The Panic Handler

- available on targets that supply `std`
  - otherwise need to provide a custom implementation with attribute `#[panic_handler]`
  - the panic handler function must never return
- it:
  - invokes the _panic hook_
  - unwinds the current thread’s stack or aborts the process

### Program Initialization

- `lang_start` function is executed at startup and in turn calls `main`
- on embedded systems, a custom implementation must be provided
  - decorated with `#[export_name = "main"]`
  - pay attention to static variable initialization

### The Out-of-Memory Handler

- it specifies what happens if that global allocator fails to allocate memory
- the default behavior of the `std` handler is to print an error message and abort the process
- without `std`, a custom handler must be provided and decorated with `#[alloc_error_handler]`

## Low-Level Memory Accesses

- _volatile_ memory operations that cannot be elided or reordered with respect to other volatile operations
  - don’t break _memory mapping_
  - interrupt handlers won’t observe out-of-order execution
- usage of inline assembly is still unstable in Rust

## Misuse-Resistant Hardware Abstraction

- In `no_std` programs, it is immensely important to use the type system to make illegal states impossible to represent
  - `PhantomData` may help

## Cross-Compilation

- programs could be built for a different _target_ than the _host_ platform
- a target triple take the form _machine-­vendor-os_
- the target platform also dictates what components of the standard library are available
